#Finetuning BART


###Scroll down to the testing part if you only want to test the finetuned model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
  
# Opening JSON file
data_path = '/content/drive/MyDrive/CSE 576 (NLP)/Project/Dataset'
f = open(data_path+'/testparser.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

In [ ]:
print(data['instance_1']['decompositions'][0])

{'question': "How many people are needed to paint Mr Khan's house?", 'answer': '10 people'}


In [ ]:
import pandas as pd
datasetdecomposition=pd.DataFrame()
datadec=[]
truths = []
for i in data:
  instancedata=[]
  instancedata.append(data[i]['context']+data[i]['original_question'])
  questionstring=''
  truths.append(data[i]['correct'])
  for q_num,j in enumerate(data[i]['decompositions']):
    # questionstring=questionstring+'Sub-problem '+str(q_num+1)+':' +j['question']+'\nAnswer: '+j['answer']+"\n"
    questionstring=questionstring+'Sub-problem '+str(q_num+1)+':' +j['question']+"\n"
  instancedata.append(questionstring)
  datadec.append(instancedata)
df=pd.DataFrame(datadec, columns=['Input', 'Output'])
df

,Input,Output
0,a constructor estimates that 10 people can pai...,Sub-problem 1:How many people are needed to pa...
1,the percentage profit earned by selling an art...,Sub-problem 1:What is the percentage profit/lo...
2,"running at the same constant rate , 6 identica...",Sub-problem 1:How many bottles can 6 machines ...
3,x% of 360 = 108. what is x?,Sub-problem 1:How much is 108 out of 360?\nSub...
4,diana is painting statues . she has 1 / 2 of a...,Sub-problem 1:How much paint does one statue r...
5,a corporation double its annual bonus to 100 o...,Sub-problem 1:What is the original annual bonu...
6,a and b together do a work in 20 days . b and ...,Sub-problem 1:How long does it take for a and ...
7,calculate the ratio between x and y if 30 % of...,Sub-problem 1:What is 30 % of x?\nSub-problem ...
8,a trader bought a car at 20 % discount on its ...,Sub-problem 1:What is the original price of th...
9,a start walking from a place at a uniform spee...,Sub-problem 1:How long did it take for b to ov...


In [ ]:
print(truths)

['d', 'e', 'c', 'a', 'a', 'a', 'b', 'b', 'c', 'b', 'b', 'c ', 'c', 'a', 'd', 'd', 'd', 'b', 'c', 'c', 'a', 'a', 'c', 'c', 'b', 'c', 'a', 'd', 'b', 'b', 'c', 'b', 'e', 'c', 'a', 'd', 'e', 'b', 'a', 'a', 'c', 'd', 'e', 'e', 'a', 'e', 'd', 'd', 'b', 'a']


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datasets
# len = len(df)
train_df = df[:int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]
train_dataset = datasets.Dataset.from_dict(train_df)
test_dataset = datasets.Dataset.from_dict(test_df)
my_dataset_dict = datasets.DatasetDict({"train":train_dataset,"test":test_dataset})

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import Trainer, TrainingArguments
from transformers.models.bart.modeling_bart import shift_tokens_right
# from transformers.modeling_bart import shift_tokens_right


tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,

In [ ]:
# This function is copied from modeling_bart.py
def shift_tokens_right(input_ids, pad_token_id):
    """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens

In [ ]:
import numpy as np
dataset = my_dataset_dict 
def convert_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Input'], pad_to_max_length=True, return_tensors='pt', max_length=128, truncation=True)
    target_encodings = tokenizer(example_batch['Output'], pad_to_max_length=True, return_tensors='pt',max_length=256, truncation=True)
    
    labels = target_encodings['input_ids']
    # labels = np.array(labels)
    decoder_input_ids = shift_tokens_right(labels, model.config.pad_token_id,)
    
    labels[labels[:, :] == model.config.pad_token_id] = -100
    labels = labels.tolist()
    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': decoder_input_ids,
        'labels': labels,
    }
    
    return encodings

dataset = dataset.map(convert_to_features, batched=True)
columns = ['input_ids','decoder_input_ids', 'labels','attention_mask',] 
dataset.set_format(type='torch', columns=columns)



  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model_path = '/content/drive/MyDrive/CSE 576 (NLP)/Project/Model'
log_path = '/content/drive/MyDrive/CSE 576 (NLP)/Project/Model'
model_name = "bart".split("/")[-1]
# args = Seq2SeqTrainingArguments(
#     f"{model_name}-finetuned-xsum",
training_args = TrainingArguments(
    f"{model_name}-finetuned-mathqa-decomposition",          
    num_train_epochs=100,           
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1,   
    warmup_steps=500,               
    weight_decay=0.01,                
    push_to_hub=True,
    hub_token = '' 
)

trainer = Trainer(
    model=model,                       
    args=training_args,                  
    train_dataset=dataset['train'],        
    eval_dataset=dataset['test']   
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/huggingface_hub/repository.py:729: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.12. Please create the repository first using `create_repo(..., exists_ok=True)`.
  FutureWarning,
Cloning https://huggingface.co/premsuresh/bart-finetuned-mathqa-decomposition into local empty directory.


In [ ]:
trainer.train()


The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: Output, Input. If Output, Input are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40
  Num Epochs = 100
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 139420416


Step,Training Loss
500,1.195300
1000,0.190500
1500,0.060900
2000,0.021200
2500,0.008700
3000,0.004100
3500,0.001600
4000,0.001300


Saving model checkpoint to bart-finetuned-mathqa-decomposition/checkpoint-500
Configuration saved in bart-finetuned-mathqa-decomposition/checkpoint-500/config.json
Model weights saved in bart-finetuned-mathqa-decomposition/checkpoint-500/pytorch_model.bin
Saving model checkpoint to bart-finetuned-mathqa-decomposition/checkpoint-1000
Configuration saved in bart-finetuned-mathqa-decomposition/checkpoint-1000/config.json
Model weights saved in bart-finetuned-mathqa-decomposition/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to bart-finetuned-mathqa-decomposition/checkpoint-1500
Configuration saved in bart-finetuned-mathqa-decomposition/checkpoint-1500/config.json
Model weights saved in bart-finetuned-mathqa-decomposition/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to bart-finetuned-mathqa-decomposition/checkpoint-2000
Configuration saved in bart-finetuned-mathqa-decomposition/checkpoint-2000/config.json
Model weights saved in bart-finetuned-mathqa-decomposition/c

TrainOutput(global_step=4000, training_loss=0.18544983218610286, metrics={'train_runtime': 627.0278, 'train_samples_per_second': 6.379, 'train_steps_per_second': 6.379, 'total_flos': 304868229120000.0, 'train_loss': 0.18544983218610286, 'epoch': 100.0})

In [ ]:
trainer.push_to_hub()
# 

Saving model checkpoint to bart-finetuned-mathqa-decomposition
Configuration saved in bart-finetuned-mathqa-decomposition/config.json
Model weights saved in bart-finetuned-mathqa-decomposition/pytorch_model.bin
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/532M [00:00<?, ?B/s]

Upload file runs/Nov29_09-26-40_b181cf3a8805/events.out.tfevents.1669714009.b181cf3a8805.2265.2:  53%|#####3  …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/premsuresh/bart-finetuned-mathqa-decomposition
   0c432a7..020ada8  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/premsuresh/bart-finetuned-mathqa-decomposition
   0c432a7..020ada8  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/premsuresh/bart-finetuned-mathqa-decomposition
   020ada8..83afbd9  main -> main

   020ada8..83afbd9  main -> main



'https://huggingface.co/premsuresh/bart-finetuned-mathqa-decomposition/commit/020ada88867bec25a6f5f56d07711045d4b66f13'

In [ ]:
# from transformers import AutoModelForSeq2SeqLM
model = BartForConditionalGeneration.from_pretrained('premsuresh/bart-finetuned-mathqa-decomposition')
# model = trainer.model
# model = AutoModelForSeq2SeqLM.from_pretrained("premsuresh/t5-small-finetuned-xsum")

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--premsuresh--bart-finetuned-mathqa-decomposition/snapshots/83afbd9e0be88ce0d3af0c61e01bcdde3f232cd6/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
 

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--premsuresh--bart-finetuned-mathqa-decomposition/snapshots/83afbd9e0be88ce0d3af0c61e01bcdde3f232cd6/pytorch_model.bin
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at premsuresh/bart-finetuned-mathqa-decomposition.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.


In [ ]:
inp="Three people are in a car and their average weight is 76 kg. Another person enters the car and the average weight of the all the people now is 82 kg. what is the weight of the 4th person?"

In [ ]:
input_ids =tokenizer(inp, return_tensors="pt", max_length=128, truncation=True).input_ids
# decoder_input_ids = tokenizer(inp, return_tensors="pt", max_length=max_len, truncation=True).input_ids  # Batch size 1
summary_ids = model.generate(input_ids, max_length=256)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'Sub-problem 1:What is the average weight of the people in the car?\nSub–problem 2:How much is the volume of the car after the people are left?\nsub-problem 3:addition ! answer of sub-problem 2 ! answer from sub- problem 1\n'

#Testing the model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
import json
  
# Opening JSON file
data_path = '/content/drive/MyDrive/CSE 576 (NLP)/Project/Dataset'
f = open(data_path+'/manualDecomposition.json')
data = json.load(f)

In [ ]:
print(data)

{'instance_1': {'context': 'a train 300 m long is running at a speed of 45 km / hr .', 'original_question': 'in what time will it pass a bridge 150 m long ?', 'options': 'a ) 40 , b ) 36 , c ) 41 , d ) 42 , e ) 34', 'decomposition': [{'question': 'what is the total length the train needs to travel to cross the bridge completely?', 'answer': '450'}, {'question': 'what is the time required to travel 450 m', 'answer': '0.01 hours'}, {'question': 'what is 0.01 hours in seconds', 'answer': '36'}]}, 'instance_2': {'context': 'a train running at the speed of 120 km / hr crosses a pole in 18 seconds . ', 'original_question': 'what is the length of the train ?', 'options': 'a ) 560 , b ) 570 , c ) 580 , d ) 590 , e ) 600', 'decomposition': [{'question': 'what is the speed of the train in m/s?', 'answer': '33.33'}, {'question': 'how much distance does the train cover in 18 seconds?', 'answer': '600'}]}, 'instance_3': {'context': 'a train 100 meters long completely crosses a 300 meters long bridg

In [ ]:
import pandas as pd
datasetdecomposition=pd.DataFrame()
datadec=[]
truths = []
for i in data:
  instancedata=[]
  instancedata.append(data[i]['context']+data[i]['original_question'])
  questionstring=''
  # truths.append(data[i]['correct'])
  for q_num,j in enumerate(data[i]['decomposition']):
    # questionstring=questionstring+'Sub-problem '+str(q_num+1)+':' +j['question']+'\nAnswer: '+j['answer']+"\n"
    questionstring=questionstring+'Sub-problem '+str(q_num+1)+':' +j['question']+"\n"
  instancedata.append(questionstring)
  datadec.append(instancedata)
df=pd.DataFrame(datadec, columns=['Input', 'Output'])
df

,Input,Output
0,a train 300 m long is running at a speed of 45...,Sub-problem 1:what is the total length the tra...
1,a train running at the speed of 120 km / hr cr...,Sub-problem 1:what is the speed of the train i...
2,a train 100 meters long completely crosses a 3...,Sub-problem 1:what is the total length the tra...
3,the lenght of a room is 5.5 m and width is 4 m...,Sub-problem 1:what is the area of the room flo...
4,a cycle is bought for rs . 900 and sold for rs...,Sub-problem 1:what is the gain for the cycle s...
5,201834 x 99999 = m find the value of m,Sub-problem 1:what is the value of 201834 mult...
6,"due to construction , the speed limit along an...",Sub-problem 1:what is the time taken to cross ...
7,there are 6 people in the elevator . their ave...,Sub-problem 1:what is total weight of 6 person...
8,two trains are moving in the same direction at...,Sub-problem 1:what is relative speed of the tw...
9,a trader sells 23 meters of cloth for rs . 529...,Sub-problem 1:how much did 1 meter of cloth se...


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 24.9 MB/s 
     |████████████████████████████████| 182 kB 65.6 MB/s 
     |████████████████████████████████| 7.6 MB 58.3 MB/s 


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('premsuresh/bart-finetuned-mathqa-decomposition')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
outputs = {}
for i,inp in enumerate(df['Input']):
  input_ids =tokenizer(inp, return_tensors="pt", max_length=128, truncation=True).input_ids
  # decoder_input_ids = tokenizer(inp, return_tensors="pt", max_length=max_len, truncation=True).input_ids  # Batch size 1
  summary_ids = model.generate(input_ids, max_length=256)
  out = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  out_list = out.split("\n")
  print(out_list)
  decom_list = []
  for qapair in out_list:
    if len(qapair.split(':'))==2:
      decom_list.append({'question':qapair.split(':')[1]})
  new_dic = {'context':inp,'original_question':"",'decomposition':decom_list}
  outputs['instance_'+str(i+1)]=new_dic

['Sub-problem 1:What is the speed of the train in m/sec?', 'Sub- problem 2:How long is the bridge?', 'sub-problem 3:How large is the total distance the train needs to pass?', ' Sub-problem 4:Time taken = division ! answer of sub-problem 2 ! answer (2 x answers of sub -problem 1', '']
['Sub-problem 1:What is the speed of the train in m/sec?', 'Sub–problem 2:How long does it take to cross the pole?', 'sub-problem 3:multiplication ! answer of sub-problem one ! answer OF sub- problem 2', '']
['Sub-problem 1:How long does the train require to cross the 300 meters long bridge?', 'Sub–problem 2:What is the speed of the train in m/sec?', 'sub-problem 3:division ! answer of sub-problem 2 ! answer from sub/problem 1', '']
['Sub-problem 1:what is the volume of the room?', 'Sub- problem 2:What is the breadth of the floor?', 'sub-problem 3:division ! answer of sub-problem 2 ! answer from sub- problem 1', '']
['Sub-problem 1:What is the difference between the selling price and cost price?', 'Sub- pr

In [ ]:
# outputs = json.dumps(outputs, indent=4, ensure_ascii=False)
print(outputs)
with open(data_path+'/bartDecompositions.json','w') as f:
    json.dump(outputs,f, indent=4, ensure_ascii=False)

{'instance_1': {'context': 'a train 300 m long is running at a speed of 45 km / hr .in what time will it pass a bridge 150 m long ?', 'original_question': '', 'decomposition': [{'question': 'What is the speed of the train in m/sec?'}, {'question': 'How long is the bridge?'}, {'question': 'How large is the total distance the train needs to pass?'}, {'question': 'Time taken = division ! answer of sub-problem 2 ! answer (2 x answers of sub -problem 1'}]}, 'instance_2': {'context': 'a train running at the speed of 120 km / hr crosses a pole in 18 seconds . what is the length of the train ?', 'original_question': '', 'decomposition': [{'question': 'What is the speed of the train in m/sec?'}, {'question': 'How long does it take to cross the pole?'}, {'question': 'multiplication ! answer of sub-problem one ! answer OF sub- problem 2'}]}, 'instance_3': {'context': 'a train 100 meters long completely crosses a 300 meters long bridge in 45 seconds . what is the speed of the train is ?', 'origina